# Vehicle detection

Plan:
- Find dataset of satellite images
- Generate images inserting there photos of vehicles
- Edit YOLOv5 to meet our needs
- Train model

In [ ]:
!git clone https://github.com/Smfun12/yolov5  # clone
%pip install -r yolov5/requirements.txt  # install
%pip install wget

In [15]:
import numpy as np
from torchvision.io import read_video
import os
import cv2
import json
import wget
import shutil

In [16]:
for i in range(28, 202):
  URL = "https://storage.googleapis.com/drone_vehicle_footage_dataset_public/video_"+ str(i) + ".zip"
  try:
      response = wget.download(URL, "/content/video_" + str(i) + ".zip")
  except Exception:
      # print('not found: ' + str(i))
      pass

In [17]:
import zipfile

zip_files = [i for i in os.listdir('/content') if '.zip' in i]
print(zip_files)
for zip_file in zip_files:
    with zipfile.ZipFile('/content/'+zip_file, 'r') as zip_ref:
        zip_ref.extractall('/content/videos'+'/video'+zip_file.split('_')[1].split('.')[0]+'/')


['video_124.zip', 'video_138.zip', 'video_177.zip', 'video_142.zip', 'video_139.zip', 'video_184.zip', 'video_92.zip', 'video_164.zip', 'video_200.zip', 'video_130.zip', 'video_182.zip', 'video_79.zip', 'video_192.zip', 'video_151.zip', 'video_161.zip', 'video_90.zip', 'video_158.zip', 'video_89.zip', 'video_48.zip', 'video_176.zip', 'video_77.zip', 'video_186.zip', 'video_153.zip', 'video_133.zip', 'video_136.zip', 'video_127.zip', 'video_150.zip', 'video_28.zip', 'video_152.zip', 'video_76.zip', 'video_157.zip', 'video_91.zip', 'video_39.zip', 'video_194.zip', 'video_160.zip', 'video_156.zip', 'video_78.zip', 'video_37.zip', 'video_159.zip', 'video_163.zip', 'video_57.zip', 'video_123.zip', 'video_155.zip', 'video_148.zip', 'video_30.zip', 'video_154.zip', 'video_93.zip', 'video_131.zip', 'video_121.zip', 'video_105.zip', 'video_180.zip', 'video_162.zip', 'video_193.zip', 'video_137.zip', 'video_106.zip', 'video_140.zip', 'video_179.zip', 'video_166.zip', 'video_40.zip', 'video_41.zi

In [18]:
zip_files = [zip_file for zip_file in os.listdir('/content/') if '.zip' in zip_file]
for zip_file in zip_files:
  os.remove('/content/' + zip_file)

In [19]:
shutil.rmtree('/content/dataset')

In [20]:
os.makedirs('/content/dataset/custom/images/train')
os.makedirs('/content/dataset/custom/labels/train')

In [21]:
videos = os.listdir('/content/videos')
classes = {'Vehicle': 0}
files_amount = 0
max_amount = 2000
condition_break = False
for video_folder in videos:
    for root, dir, files in os.walk('/content/videos/'+video_folder):
        if 'annotations.json' not in files:
            continue
        idx = files.index('annotations.json')

        with open('/content/videos/'+video_folder+'/'+files[idx], 'r') as json_file:
            json_file = json.load(json_file)
        tracks = json_file[0]['tracks']
        vehicles = [vehicle for vehicle in tracks if vehicle['label'] == 'Vehicle']
        success_frames = []
        points_arr = []
        for i in vehicles:
            for j in i['shapes']:
                if j['points'] is not None and j['frame'] not in success_frames:
                    success_frames.append(j['frame'])
                    points_arr.append(j['points'])
                    files_amount += 1
                    if files_amount == max_amount:
                        condition_break = True
                        break
            if condition_break:
                break

        video = os.listdir('/content/videos/'+video_folder+'/'+dir[0])[0]
        vidcap = cv2.VideoCapture('/content/videos/'+video_folder+'/'+dir[0]+'/'+video)
        success, image = vidcap.read()
        count = 0
        shapes = {}
        while success:
            if count in success_frames:
                shapes[count] = (image.shape[1], image.shape[0])
                cv2.imwrite("/content/dataset/custom/images/train/frame%d.jpg" % count, image)  # save frame as JPEG file
            success, image = vidcap.read()
            count += 1
        counter = 0
        assert len(shapes) == len(success_frames), 'Len shapes is less that frames'
        for points in points_arr:
            success_frame = success_frames[counter]
            x_center = (points[2] + points[0]) / 2
            y_center = (points[3] + points[1]) / 2
            width = abs(points[2] - points[0])
            height = abs(points[3] - points[1])
            img_width = shapes[success_frame][0]
            img_height = shapes[success_frame][1]
            x_center_normalized = min(1.0,x_center / img_width)
            y_center_normalized = min(1.0, y_center / img_height)
            width_normalized = min(1.0, width / img_width)
            height_normalized = min(1.0,height / img_height)
            # assert x_center_normalized < 1.0
            # assert y_center_normalized < 1.0
            # assert width_normalized < 1.0
            # assert height_normalized < 1.0
            file = open('/content/dataset/custom/labels/train/frame' + str(success_frame) + '.txt', 'w')
            yolo_format = str(classes['Vehicle']) + " " + str(x_center_normalized) + " " + str(
                y_center_normalized) + " " + str(width_normalized) + " " + str(height_normalized)
            file.write(yolo_format)
            file.close()
            counter += 1
    if condition_break:
        break


In [ ]:
# Train YOLOv5s on dataset for 3 epochs
# %cd yolov5
!python /content/yolov5/train.py --img 640 --batch 16 --epochs 10 --data custom_dataset.yaml --weights yolov5s.pt

In [25]:
shutil.make_archive('result', 'zip', '/content/yolov5/runs')

'/content/result.zip'